# **PDF Image Extractor and Classification**<br/>
**Master's Degree in Data Science (A.Y. 2023/2024)**<br/>
**University of Milano - Bicocca**<br/>

Vittorio Haardt, Luca Porcelli

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Installazione pacchetti e caricamento librerie

In [ ]:
pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 39.0 MB/s eta 0:00:00


In [ ]:
pip install pytesseract opencv-python

In [ ]:
!sudo apt-get install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 31 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 3s (1,799 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [ ]:
import fitz  # PyMuPDF
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import data
from skimage import color
from skimage import morphology
from skimage import segmentation
import copy
from scipy.spatial import ConvexHull
import pytesseract
import tqdm
from PIL import Image
import math
import keras
import tensorflow as tf

# Funzione di Estrazione

## Funzioni di supporto

### Pagine PDF

La funzione `pdf_to_images` per estrarre una lista contenente ogni pagina del pdf come immagine (np.array)

In [ ]:
def pdf_to_images(pdf_path, resolution=300, color_format="rgb"):
    pdf_document = fitz.open(pdf_path)
    images = []
    for page_number in range(pdf_document.page_count):
        page = pdf_document.load_page(page_number)
        pixmap = page.get_pixmap(matrix=fitz.Matrix(resolution/72.0, resolution/72.0), clip=page.rect, colorspace=color_format)
        img_array = np.frombuffer(pixmap.samples, dtype=np.uint8).reshape((pixmap.height, pixmap.width, 3))
        images.append(img_array)
    pdf_document.close()
    return images

### Maschere

La funzione `mask_calculation` che prende un'immagine di una pagina PDF come input e la elabora per creare una maschera booleana in cui i pixel con un'intensità luminosa superiore a 0.99 vengono considerati parte dell'oggetto principale.

Rimuove eventuali piccoli dettagli o oggetti che potrebbero essere indesiderati, basandosi su una dimensione minima di 500 pixel. La maschera viene ulteriormente elaborata per rimuovere i buchi al suo interno.

Viene eseguita un'operazione di apertura sulla maschera. L'apertura è una combinazione di erosione seguita da dilatazione. Un elemento strutturante circolare con un raggio di 3 pixel viene utilizzato durante questa operazione. Serve a eliminare ulteriori dettagli e a garantire una forma più uniforme dell'oggetto principale.

La funzione restituisce la maschera elaborata.

[[Source](https://scikit-image.org/docs/stable/auto_examples/segmentation/plot_mask_slic.html)]

In [ ]:
def mask_calculation(pdf_page):
  lum = color.rgb2gray(pdf_page)
  mask = morphology.remove_small_holes(morphology.remove_small_objects(lum > 0.99, 500),500)
  mask = morphology.opening(mask, morphology.disk(3))
  return mask

### Divisione maschere

La funzione `mask_separation` è progettata per dividere una maschera complessa in maschere più semplici attraverso tre fasi di ricerca:

- Ricerca Orizzontale Iniziale:
Divide le maschere a diverse altezze rispetto alla maschera principale.
- Ricerca Verticale:
Suddivide le maschere generate nella fase precedente in maschere con posizioni laterali diverse.
- Ricerca Orizzontale Aggiuntiva per Pulizia:
Esegue un altro ciclo di divisione orizzontale per pulire ulteriormente le maschere risultanti.

Durante l'esecuzione della funzione, le maschere troppo piccole vengono eliminate

In [ ]:
def mask_separation(img, mask):
  maskere = []
  maskere_fin = []
  dim_img = img.shape[0]*img.shape[1]
  while not np.all(mask):
      found_all_true = False
      first_false = False

      mask2 = copy.deepcopy(mask)
      for i, row in enumerate(mask2):
          if False in row and not first_false:
              first_false = True
          elif first_false:
              if False not in row:
                  mask2[i] = [True] * len(row)
                  found_all_true = True
              elif not found_all_true:
                  pass
              else:
                  mask2[i] = [True] * len(row)

      maskere.append(mask2)
      mask = mask2 ^ mask
      mask = ~mask

  for m in maskere:
    mask_t = m.T
    while not np.all(mask_t):
        found_all_true = False
        first_false = False

        mask2 = copy.deepcopy(mask_t)

        for i, row in enumerate(mask2):
            if False in row and not first_false:
                first_false = True
            elif first_false:
                if False not in row:
                    mask2[i] = [True] * len(row)
                    found_all_true = True
                elif not found_all_true:
                    pass
                else:
                    mask2[i] = [True] * len(row)

        if np.count_nonzero(mask2.T == False) > dim_img/50:
          maskere_fin.append(mask2.T)

        mask_t = np.logical_xor(mask2, mask_t)
        mask_t = ~mask_t

  maskere_fin_pul = []
  for ma in maskere_fin:
      mask_t = ma
      dim_img = img.shape[0] * img.shape[1]

      while not np.all(mask_t):
          found_all_true = False
          first_false = False

          mask2 = copy.deepcopy(mask_t)

          for i, row in enumerate(mask2):
              if False in row and not first_false:
                  first_false = True
              elif first_false:
                  if False not in row:
                      mask2[i] = [True] * len(row)
                      found_all_true = True
                  elif not found_all_true:
                      pass
                  else:
                      mask2[i] = [True] * len(row)

          if np.count_nonzero(mask2.T == False) > dim_img / 100:
              maskere_fin_pul.append(mask2)

          mask_t = np.logical_xor(mask2, mask_t)
          mask_t = ~mask_t
  return maskere_fin_pul

### Raffinamento maschere ed estrazione immagini

La Funzione`single_img`, estrae regioni specifiche da un'immagine di input sulla base di una lista di maschere. Vediamo una spiegazione dettagliata:

La funzione itera sulla lista di maschere fornita come input.
Per ogni maschera:
     - Trova i contorni della maschera utilizzando la funzione `cv2.findContours`.
     - Estrae i vertici convessi dell'inviluppo convesso dei contorni trovati.
     - Calcola i valori minimi e massimi dei vertici convessi per definire un rettangolo delimitatore, con `ConvexHull`.
     - Esegue il ritaglio dell'immagine di input utilizzando il rettangolo delimitatore.

Viene calcolata la dimensione della regione ritagliata e confrontata con la dimensione totale dell'immagine iniziale. Solo se la dimensione della regione ritagliata è maggiore dell'1% della dimensione totale dell'immagine iniziale.

La funzione restituisce una lista contenente le immagini estratte sulla base delle maschere fornite.

In [ ]:
def single_img(immagine, lista_maschere):
  single_images = []
  dim_img = immagine.shape[0]*immagine.shape[1]
  for i in lista_maschere:
    contours, _ = cv2.findContours(i.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    corners = contours[1].reshape(-1, 2)
    hull = ConvexHull(corners)
    convex_hull_vertices = corners[hull.vertices]
    min_x, min_y = np.min(corners, axis=0)
    max_x, max_y = np.max(corners, axis=0)
    cropped_image = immagine[min_y:max_y, min_x:max_x]
    dimensione = cropped_image.shape[0]*cropped_image.shape[1]
    if dimensione > dim_img / 100:
      single_images.append(cropped_image)
  return single_images

### Rimozione testi

La funzione `filter_text` filtra un elenco di array di immagini sulla base della quantità di colori unici presenti. Ecco una spiegazione dettagliata:

Per ciascun array di immagine nella lista:
     Trova i colori unici nell'immagine.
     - Valuta se il numero di colori unici è inferiore a 10000 e memorizza il risultato.
     - Aggiunge il valore booleano ad una lista.

Viene creata una lista contenente solo le immagini con più di 10000 colori unici.

In [ ]:
def filter_text(lista_array):
  if not lista_array:
    return []
  list_ok = []
  for im in lista_array:
    if len(im.shape) == 3:
      im = im.reshape((-1, 3))
    colori_unici = np.unique(im, axis=0)
    value = len(colori_unici) < 10000
    list_ok.append(value)
    output_images = [valore for valore, include in zip(lista_array, list_ok) if not include]
  return output_images


## Funzione di Estrazione

La funzione `images_from_pdf` incorpora le funzioni precedenti per estrarre efficacemente le immagini da un pdf.

In [ ]:
def images_from_pdf(pdf_path):
  EXTRACTED_IMGS = []
  #estraggo immagine per ogni pagina
  pdf_images = pdf_to_images(pdf_path)
  # per ogin pagina
  for im in tqdm.tqdm(pdf_images, desc='Processed PDF pages'):
    #dimensione pagina
    dim_im = im.shape[0]*im.shape[1]

    #calcolo maschera
    mask = mask_calculation(im)

    # separo in piu maschere divise
    li_masks = mask_separation(im, mask)

    #applico le maschere
    li_imgs = single_img(im, li_masks)

    #tologo le immagini con solo testo
    imgs_no_text = filter_text(li_imgs)

    #aggiungo alla lista
    for i in imgs_no_text:
      EXTRACTED_IMGS.append(i)
  return EXTRACTED_IMGS

## Estrazione Immagini

Applicazione della funzione di estrazione delle immagini sul PDF.

In [ ]:
immagini_estratte = images_from_pdf('/content/drive/MyDrive/VIPM/PDF part/food_text.pdf')

Processed PDF pages: 100%|██████████| 21/21 [06:13<00:00, 17.78s/it]


Visualizzazione delle immagini estratte.

In [ ]:
num_masked_images = len(immagini_estratte)
num_cols = 4
num_rows = math.ceil(num_masked_images / num_cols)
fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 5*num_rows))
axes = axes.flatten()
for i, ax in enumerate(axes[:num_masked_images]):
    ax.imshow(immagini_estratte[i])
for j in range(num_masked_images, len(axes)):
    fig.delaxes(axes[j])
plt.show()

Output hidden; open in https://colab.research.google.com to view.

# Classificazione Immagini

Caricamento modello previsionale.

In [ ]:
food_nonfood = keras.models.load_model('/content/drive/MyDrive/VIPM/Models/Model_PDF.Keras')

Visualizzazione risultati di classficazione.

In [ ]:
num_masked_images = len(immagini_estratte)
num_cols = 4
num_rows = math.ceil(num_masked_images / num_cols)
fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 5*num_rows))
axes = axes.flatten()
for i, ax in enumerate(axes[:num_masked_images]):
    resized_image = Image.fromarray(immagini_estratte[i]).resize((224, 224))
    image_array = tf.keras.preprocessing.image.img_to_array(resized_image)
    image_array = tf.expand_dims(image_array, 0)  # Aggiungi una dimensione per la batch
    # Effettua la previsione
    predictions = food_nonfood.predict(image_array)
    # Interpreta i risultati
    predicted_class = tf.argmax(predictions[0]).numpy()

    ax.imshow(immagini_estratte[i])
    if predicted_class == 0:
        pred = "Food"
        title_color = "darkgreen"  # Colore verde scuro per Food
    else:
        pred = "Non-Food"
        title_color = "blue"  # Colore blu per Non-Food
    ax.set_title(pred, color=title_color)  # Imposta il colore del titolo
for j in range(num_masked_images, len(axes)):
    fig.delaxes(axes[j])
plt.show()


Output hidden; open in https://colab.research.google.com to view.